Calibrating a logistic regression model for the probability of credit default

In [1]:
!pip install statsmodels
!pip install xlrd

You are using pip version 9.0.3, however version 20.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.3, however version 20.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
#Import necessary packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.metrics import classification_report,confusion_matrix

In [3]:
#Load Data
CreditData = pd.read_excel("CreditData2020.xls") 

# have a look at the data headers print(CreditData.head(0))
# get basic descriptive statistics of the data print(CreditData.describe())

In [4]:
CreditData

,No,Status,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan
0,1,0,5,other,23,female,single,no,fixed,35,120,0,0,1300,5.0
1,2,0,23,rent,47,male,single,no,fixed,44,86,0,0,350,2.0
2,3,0,19,other,55,male,married,no,fixed,45,118,4000,0,1300,5.0
3,4,0,12,rent,37,female,married,yes,fixed,86,110,0,0,700,5.0
4,5,0,3,other,21,female,single,yes,fixed,35,75,0,0,500,1.5
5,6,0,0,other,31,male,single,no,partime,35,65,2500,0,1000,4.0
6,7,1,20,rent,47,female,married,no,fixed,85,100,8000,0,1250,5.0
7,8,0,15,owner,42,male,single,no,fixed,35,90,5000,3000,1000,3.0
8,9,0,3,owner,27,female,married,no,fixed,35,120,4000,1500,1800,5.0
9,10,0,20,owner,41,female,married,no,freelance,90,250,13500,0,1900,3.0


In [5]:
# let’s choose about 2/3 (0.67%) of the data for learning, and the rest # for testing the model
# split data into training and testing sets
nData = len(CreditData.index)
nTrain = round(nData*0.67) 
CreditDataTrain = CreditData[:nTrain] 
CreditDataTest = CreditData[nTrain:]

In [6]:
# estimate logistic regression model on the training data 

model = smf.logit(formula='Status ~ LengthOfEmployment + Home + Age + Sex + MaritalStatus + Records + Job + Expenses + Income + Assets + Debt + LoanAmount + DurationOfLoan', data = CreditDataTrain).fit()

# results of model fit
print(model.summary())
print(model.params) 
print(model.pvalues)

Optimization terminated successfully.
         Current function value: 0.453889
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Status   No. Observations:                 2979
Model:                          Logit   Df Residuals:                     2960
Method:                           MLE   Df Model:                           18
Date:                Sun, 06 Dec 2020   Pseudo R-squ.:                  0.2373
Time:                        15:37:46   Log-Likelihood:                -1352.1
converged:                       True   LL-Null:                       -1772.9
                                        LLR p-value:                4.710e-167
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.5299      0.384     -3.982      0.000      -2.283

In [7]:
#Run and store wald_test on the whole dataset
wald_values = model.wald_test_terms().table
wald_values

,statistic,pvalue,df_constraint
Intercept,15.856057,6.834743006936223e-05,1
Home,39.047532,3.3184564535793915e-09,2
Sex,0.073828,0.7858425557456905,1
MaritalStatus,27.787394,4.025033691799671e-06,3
Records,194.410018,3.4655790501962964e-44,1
Job,95.590799,1.378300291652316e-20,3
LengthOfEmployment,81.125949,2.117807661504381e-19,1
Age,5.037043,0.02481081087300651,1
Expenses,29.437458,5.775057735666685e-08,1
Income,76.916163,1.7837130152681534e-18,1


In [8]:
#Comments
#the R squared is around 23%, which is not very high but still able to describe almost 1/4 of the total variance
#looking at the p-values, we can observe that the dummy variable "renter" (tenant), is not a statistically significant variable. 
#hence, being a tenant seems not influencing the ability to repay the loan of the individual
#however, being an owner seems instead being able to influence the ability of the individual to repay, 
#We must also point out that the home category "other" has not been included in order to avoid the multicollinearity problem

#Also the "Sex" dummy variable seems not able to influence the ability to repay of the individual, given its very high p-value

#Marital Status: while both being "Single" or "Married" seems influencing the probability to repay the loan, being "widow" has a not statistically significant coefficient

#Surprisingly, "Age" has a p-value of 2.5%, which is significant at a 95% cf. level but not at 99%. So we will remove it from our regression

#Finally, also the "Duration of the loan" seems not influencing the probability of repaying the loan

####COMMENTS ON THE COEFFICIENTS!!!
#if coefficient is positive, it means that the variable influences positively (increases) the probability of default
#explain why those variables have certain coefficients
#e.g. "Income"
#income has a negative coefficient so it influences negatively (decreases) the probability of default of the loan.
#we could easily expect that, the higher the income, more resources has the individual to pay the coupons etc.




In [9]:
#we remove all the regressors which are not significant at least at the 99% cf. level (missing the removal of the dummy variables)
model = smf.logit(formula='Status ~ LengthOfEmployment + Home  + MaritalStatus+ Records + Job + Expenses + Income + Assets + Debt + LoanAmount', data = CreditDataTrain).fit()

# results of model fit
print(model.summary())
print(model.params) 
print(model.pvalues)

Optimization terminated successfully.
         Current function value: 0.455223
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Status   No. Observations:                 2979
Model:                          Logit   Df Residuals:                     2963
Method:                           MLE   Df Model:                           15
Date:                Sun, 06 Dec 2020   Pseudo R-squ.:                  0.2351
Time:                        15:37:47   Log-Likelihood:                -1356.1
converged:                       True   LL-Null:                       -1772.9
                                        LLR p-value:                5.876e-168
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.8478      0.295     -2.874      0.004      -1.426

In [10]:
#we can observe only a very slight increase in the R squared of our model

In [11]:
#Re-run the wald_test, but only on the cleaned data
wald_values = model.wald_test_terms().table
wald_values

,statistic,pvalue,df_constraint
Intercept,8.257521,0.00405835187705181,1
Home,37.300654,7.948148713768364e-09,2
MaritalStatus,27.579513,4.450269159390369e-06,3
Records,196.244987,1.37815429686104e-44,1
Job,97.711565,4.824969965765579e-21,3
LengthOfEmployment,76.794556,1.8969976436611555e-18,1
Expenses,31.027943,2.5434024650110822e-08,1
Income,78.321718,8.755396239937548e-19,1
Assets,14.290615,0.00015664388465902682,1
Debt,19.099652,1.2406749344168264e-05,1


In [12]:
#Making predictions on the test data
prediction = model.predict(CreditDataTest)

In [13]:
CreditDataTest['Prediction']= prediction

C:\Users\hp\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
#Mapping probability greater than 50% to default (1) and otherwise to no default (0)
Prediction_1 = CreditDataTest['Prediction']>0.5
CreditDataTest['DefaultPrediction'] = Prediction_1

C:\Users\hp\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [15]:
CreditDataTest 

,No,Status,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan,Prediction,DefaultPrediction
2979,2980,0,5,owner,34,female,married,no,fixed,45,137,20000,0,3000,3.0,0.320128,False
2980,2981,0,10,other,36,male,married,no,fixed,90,172,3000,0,1100,5.0,0.194387,False
2981,2982,0,12,owner,28,male,married,no,fixed,45,148,3500,144,1350,5.0,0.073184,False
2982,2983,0,35,owner,55,female,married,no,freelance,60,84,13000,0,1390,4.0,0.042300,False
2983,2984,0,10,owner,40,male,married,no,freelance,90,301,2000,0,1100,5.0,0.094711,False
2984,2985,0,11,rent,29,female,single,no,fixed,44,93,0,0,700,1.0,0.153646,False
2985,2986,1,0,owner,64,male,widow,no,others,35,100,8000,0,1980,5.0,0.635568,True
2986,2987,1,3,owner,26,female,single,no,freelance,35,169,60000,3000,800,3.0,0.041780,False
2987,2988,1,20,owner,43,female,married,yes,fixed,45,120,10000,0,2000,5.0,0.315875,False
2988,2989,0,26,owner,55,female,married,no,fixed,60,110,10000,0,1500,5.0,0.042994,False


In [16]:
CreditDataTest['DefaultPrediction'] = CreditDataTest['DefaultPrediction']*1

C:\Users\hp\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
CreditDataTest

,No,Status,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan,Prediction,DefaultPrediction
2979,2980,0,5,owner,34,female,married,no,fixed,45,137,20000,0,3000,3.0,0.320128,0
2980,2981,0,10,other,36,male,married,no,fixed,90,172,3000,0,1100,5.0,0.194387,0
2981,2982,0,12,owner,28,male,married,no,fixed,45,148,3500,144,1350,5.0,0.073184,0
2982,2983,0,35,owner,55,female,married,no,freelance,60,84,13000,0,1390,4.0,0.042300,0
2983,2984,0,10,owner,40,male,married,no,freelance,90,301,2000,0,1100,5.0,0.094711,0
2984,2985,0,11,rent,29,female,single,no,fixed,44,93,0,0,700,1.0,0.153646,0
2985,2986,1,0,owner,64,male,widow,no,others,35,100,8000,0,1980,5.0,0.635568,1
2986,2987,1,3,owner,26,female,single,no,freelance,35,169,60000,3000,800,3.0,0.041780,0
2987,2988,1,20,owner,43,female,married,yes,fixed,45,120,10000,0,2000,5.0,0.315875,0
2988,2989,0,26,owner,55,female,married,no,fixed,60,110,10000,0,1500,5.0,0.042994,0


In [18]:
#Checking the accuracy of predictions on the test data
Accuracy0 = (CreditDataTest['Status']== CreditDataTest['DefaultPrediction'])
Accuracy0 = sum(Accuracy0)
Accuracy = Accuracy0/len(CreditDataTest)
print('The Accuracy is', Accuracy)

The Accuracy is 0.799591002045


In [19]:
#Confusion Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(CreditDataTest['Status'],CreditDataTest['DefaultPrediction']))
print('\nClassification Report\n')
print(classification_report(CreditDataTest['Status'],CreditDataTest['DefaultPrediction']))

Confusion Matrix
[[955 104]
 [190 218]]

Classification Report

             precision    recall  f1-score   support

          0       0.83      0.90      0.87      1059
          1       0.68      0.53      0.60       408

avg / total       0.79      0.80      0.79      1467



In [20]:
## Loans approval
#Loading the data
CreditApplicants = pd.read_excel("CreditApplicants2020.xls") 

In [21]:
CreditApplicants

,ApplicantNo,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan
0,1,0,owner,28,female,married,no,fixed,45,80,1500,0,1260,4.0
1,2,21,other,51,male,married,no,fixed,45,140,3000,0,1200,5.0
2,3,0,rent,37,female,married,no,others,88,130,0,0,250,1.0
3,4,12,owner,64,female,single,yes,freelance,60,341,4500,0,900,3.0
4,5,2,owner,33,female,married,no,freelance,75,220,16000,500,3000,3.0
5,6,3,owner,30,male,divorced,no,fixed,60,157,15000,0,1000,5.0
6,7,9,rent,40,female,single,no,fixed,44,91,0,0,1000,5.0
7,8,4,other,50,female,married,no,freelance,60,150,7000,0,900,3.5


In [22]:
#Predicting the probablity of default
applicants_prediction = model.predict(CreditApplicants)
CreditApplicants['Default probabilities']= applicants_prediction

In [23]:
#Approving loans where default probability according to the model is below 25%
ApproveTF = CreditApplicants['Default probabilities'] < 0.25
CreditApplicants['Approve T/F']= ApproveTF
CreditApplicants

,ApplicantNo,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan,Default probabilities,Approve T/F
0,1,0,owner,28,female,married,no,fixed,45,80,1500,0,1260,4.0,0.226205,True
1,2,21,other,51,male,married,no,fixed,45,140,3000,0,1200,5.0,0.063393,True
2,3,0,rent,37,female,married,no,others,88,130,0,0,250,1.0,0.501565,False
3,4,12,owner,64,female,single,yes,freelance,60,341,4500,0,900,3.0,0.170451,True
4,5,2,owner,33,female,married,no,freelance,75,220,16000,500,3000,3.0,0.570603,False
5,6,3,owner,30,male,divorced,no,fixed,60,157,15000,0,1000,5.0,0.238654,True
6,7,9,rent,40,female,single,no,fixed,44,91,0,0,1000,5.0,0.225956,True
7,8,4,other,50,female,married,no,freelance,60,150,7000,0,900,3.5,0.265615,False


In [24]:
#Displaying loans that will be rejected
CreditApplicants[CreditApplicants['Approve T/F'] == 0]

,ApplicantNo,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan,Default probabilities,Approve T/F
2,3,0,rent,37,female,married,no,others,88,130,0,0,250,1.0,0.501565,False
4,5,2,owner,33,female,married,no,freelance,75,220,16000,500,3000,3.0,0.570603,False
7,8,4,other,50,female,married,no,freelance,60,150,7000,0,900,3.5,0.265615,False


In [25]:
#Only applicants no. 3, 5 and 8 will see their loan application refused.